In [68]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import tree
from scipy import stats

In [4]:
skin_data = pd.read_csv('Skin_NonSkin.txt', sep="\t", header=None)
skin_data

,0,1,2,3
0,74,85,123,1
1,73,84,122,1
2,72,83,121,1
3,70,81,119,1
4,70,81,119,1
...,...,...,...,...
245052,163,162,112,2
245053,163,162,112,2
245054,163,162,112,2
245055,163,162,112,2


In [5]:
skin_data.columns = ['B', 'G', 'R', 'decision']
skin_data = skin_data.dropna()
skin_data[:10]

,B,G,R,decision
0,74,85,123,1
1,73,84,122,1
2,72,83,121,1
3,70,81,119,1
4,70,81,119,1
5,69,80,118,1
6,70,81,119,1
7,70,81,119,1
8,76,87,125,1
9,76,87,125,1


In [6]:
def summerize_data(df):
    for column in df.columns:
        print(column)
        if df.dtypes[column] == np.object:
            print(df[column].value_counts())
        else:
            print(df[column].describe())
        print('\n')

In [7]:
summerize_data(skin_data)

B
count    245057.000000
mean        125.065446
std          62.255653
min           0.000000
25%          68.000000
50%         139.000000
75%         176.000000
max         255.000000
Name: B, dtype: float64


G
count    245057.000000
mean        132.507327
std          59.941197
min           0.000000
25%          87.000000
50%         153.000000
75%         177.000000
max         255.000000
Name: G, dtype: float64


R
count    245057.000000
mean        123.177151
std          72.562165
min           0.000000
25%          70.000000
50%         128.000000
75%         164.000000
max         255.000000
Name: R, dtype: float64


decision
count    245057.000000
mean          1.792461
std           0.405546
min           1.000000
25%           2.000000
50%           2.000000
75%           2.000000
max           2.000000
Name: decision, dtype: float64




In [8]:
skin_data.loc[skin_data['decision'] == 1, 'decision'] = 0
skin_data.loc[skin_data['decision'] == 2, 'decision'] = 1
skin_data[:10]

,B,G,R,decision
0,74,85,123,0
1,73,84,122,0
2,72,83,121,0
3,70,81,119,0
4,70,81,119,0
5,69,80,118,0
6,70,81,119,0
7,70,81,119,0
8,76,87,125,0
9,76,87,125,0


In [18]:
skin_data = skin_data.dropna()
skin_data

,B,G,R,decision
0,74,85,123,0
1,73,84,122,0
2,72,83,121,0
3,70,81,119,0
4,70,81,119,0
...,...,...,...,...
245052,163,162,112,1
245053,163,162,112,1
245054,163,162,112,1
245055,163,162,112,1


In [19]:
skin_data2 = skin_data.drop(['decision'], axis=1)

In [20]:
skin_data2

,B,G,R
0,74,85,123
1,73,84,122
2,72,83,121
3,70,81,119
4,70,81,119
...,...,...,...
245052,163,162,112
245053,163,162,112
245054,163,162,112
245055,163,162,112


In [14]:
skin_data['decision']

0         0
1         0
2         0
3         0
4         0
         ..
245052    1
245053    1
245054    1
245055    1
245056    1
Name: decision, Length: 245057, dtype: int64

In [22]:
skin_data.dropna()

,B,G,R,decision
0,74,85,123,0
1,73,84,122,0
2,72,83,121,0
3,70,81,119,0
4,70,81,119,0
...,...,...,...,...
245052,163,162,112,1
245053,163,162,112,1
245054,163,162,112,1
245055,163,162,112,1


In [83]:
#Random Forest
for i in range(5):
    print('trial', i+1)
    skin_rf_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
        X_train = pd.DataFrame(X_train, columns=X_train.columns)
        X_test = X_test

        for k in range(5):
            skin_rf = RandomForestClassifier(criterion='entropy')
            param_grid = [{'n_estimators': [1024], 'max_features': [1, 2, 3]}]
            #n_list = [1, 2 , 4 , 6, 8, 12, 16, 20]
            skin_rf_grid = GridSearchCV(skin_rf, param_grid, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            skin_rf_grid.fit(X_train, y_train)
            print('mean test accuracy:', skin_rf_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auc ovr:', skin_rf_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', skin_rf_grid.cv_results_['mean_test_f1_micro'])
            skin_rf_results = pd.DataFrame(skin_rf_grid.cv_results_['params'])
            skin_rf_results['score_acc'] = skin_rf_grid.cv_results_['mean_test_accuracy']
            skin_rf_list.append(skin_rf_grid.cv_results_['mean_test_accuracy'])
            skin_rf_cols = skin_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            skin_rf_results.columns = skin_rf_cols
            print(skin_rf_results)

trial 1
mean test accuracy: [0.9988 0.998  0.9974]
mean test roc auc ovr: [0.99988041 0.99986585 0.99929645]
mean test f1 micro: [0.9988 0.998  0.9974]
   max_features  n_estimators  score_acc
0             1          1024     0.9988
1             2          1024     0.9980
2             3          1024     0.9974
mean test accuracy: [0.9982 0.9976 0.9974]
mean test roc auc ovr: [0.99987313 0.99986585 0.99929463]
mean test f1 micro: [0.9982 0.9976 0.9974]
   max_features  n_estimators  score_acc
0             1          1024     0.9982
1             2          1024     0.9976
2             3          1024     0.9974
mean test accuracy: [0.9984 0.9974 0.9974]
mean test roc auc ovr: [0.99988891 0.9998877  0.99929099]
mean test f1 micro: [0.9984 0.9974 0.9974]
   max_features  n_estimators  score_acc
0             1          1024     0.9984
1             2          1024     0.9974
2             3          1024     0.9974
mean test accuracy: [0.9982 0.9976 0.9976]
mean test roc auc ovr: [0

mean test accuracy: [0.9968 0.9962 0.9954]
mean test roc auc ovr: [0.99984948 0.99985445 0.99976491]
mean test f1 micro: [0.9968 0.9962 0.9954]
   max_features  n_estimators  score_acc
0             1          1024     0.9968
1             2          1024     0.9962
2             3          1024     0.9954
mean test accuracy: [0.9964 0.9962 0.9954]
mean test roc auc ovr: [0.99986255 0.99986314 0.99973818]
mean test f1 micro: [0.9964 0.9962 0.9954]
   max_features  n_estimators  score_acc
0             1          1024     0.9964
1             2          1024     0.9962
2             3          1024     0.9954
mean test accuracy: [0.996  0.9962 0.9954]
mean test roc auc ovr: [0.99986381 0.99985506 0.99974878]
mean test f1 micro: [0.996  0.9962 0.9954]
   max_features  n_estimators  score_acc
0             1          1024     0.9960
1             2          1024     0.9962
2             3          1024     0.9954
mean test accuracy: [0.9958 0.9962 0.9956]
mean test roc auc ovr: [0.9999155

mean test accuracy: [0.9962 0.9952 0.9946]
mean test roc auc ovr: [0.99973515 0.99970593 0.99873356]
mean test f1 micro: [0.9962 0.9952 0.9946]
   max_features  n_estimators  score_acc
0             1          1024     0.9962
1             2          1024     0.9952
2             3          1024     0.9946
mean test accuracy: [0.9978 0.997  0.9958]
mean test roc auc ovr: [0.9999413  0.99990981 0.99969249]
mean test f1 micro: [0.9978 0.997  0.9958]
   max_features  n_estimators  score_acc
0             1          1024     0.9978
1             2          1024     0.9970
2             3          1024     0.9958
mean test accuracy: [0.9976 0.997  0.996 ]
mean test roc auc ovr: [0.99994858 0.99989953 0.99970037]
mean test f1 micro: [0.9976 0.997  0.996 ]
   max_features  n_estimators  score_acc
0             1          1024     0.9976
1             2          1024     0.9970
2             3          1024     0.9960
mean test accuracy: [0.9976 0.9972 0.9962]
mean test roc auc ovr: [0.9999455

mean test accuracy: [0.996  0.9962 0.9962]
mean test roc auc ovr: [0.9999006  0.99992882 0.99986378]
mean test f1 micro: [0.996  0.9962 0.9962]
   max_features  n_estimators  score_acc
0             1          1024     0.9960
1             2          1024     0.9962
2             3          1024     0.9962
mean test accuracy: [0.9962 0.9962 0.9962]
mean test roc auc ovr: [0.99990919 0.99992576 0.99987851]
mean test f1 micro: [0.9962 0.9962 0.9962]
   max_features  n_estimators  score_acc
0             1          1024     0.9962
1             2          1024     0.9962
2             3          1024     0.9962
mean test accuracy: [0.996  0.9962 0.996 ]
mean test roc auc ovr: [0.99991042 0.9999233  0.99986317]
mean test f1 micro: [0.996  0.9962 0.996 ]
   max_features  n_estimators  score_acc
0             1          1024     0.9960
1             2          1024     0.9962
2             3          1024     0.9960
mean test accuracy: [0.9958 0.9962 0.9962]
mean test roc auc ovr: [0.9999067

mean test accuracy: [0.9966 0.9966 0.9966]
mean test roc auc ovr: [0.99980727 0.9998874  0.99986155]
mean test f1 micro: [0.9966 0.9966 0.9966]
   max_features  n_estimators  score_acc
0             1          1024     0.9966
1             2          1024     0.9966
2             3          1024     0.9966
mean test accuracy: [0.9964 0.9966 0.9962]
mean test roc auc ovr: [0.99982806 0.99988802 0.99988362]
mean test f1 micro: [0.9964 0.9966 0.9962]
   max_features  n_estimators  score_acc
0             1          1024     0.9964
1             2          1024     0.9966
2             3          1024     0.9962
mean test accuracy: [0.9972 0.9964 0.996 ]
mean test roc auc ovr: [0.99997646 0.99994527 0.99938498]
mean test f1 micro: [0.9972 0.9964 0.996 ]
   max_features  n_estimators  score_acc
0             1          1024     0.9972
1             2          1024     0.9964
2             3          1024     0.9960
mean test accuracy: [0.997  0.9964 0.996 ]
mean test roc auc ovr: [0.9999788

In [84]:
skin_rf_list

[array([0.9972, 0.9964, 0.9952]),
 array([0.9972, 0.996 , 0.9948]),
 array([0.9974, 0.9958, 0.995 ]),
 array([0.9972, 0.9962, 0.995 ]),
 array([0.9972, 0.996 , 0.995 ]),
 array([0.9966, 0.9966, 0.9964]),
 array([0.9964, 0.996 , 0.9964]),
 array([0.9964, 0.9966, 0.9964]),
 array([0.9966, 0.9966, 0.9966]),
 array([0.9964, 0.9966, 0.9962]),
 array([0.9972, 0.9964, 0.996 ]),
 array([0.997 , 0.9964, 0.996 ]),
 array([0.9972, 0.9964, 0.996 ]),
 array([0.9972, 0.9962, 0.9962]),
 array([0.9972, 0.9964, 0.996 ]),
 array([0.9966, 0.9972, 0.997 ]),
 array([0.9964, 0.9974, 0.997 ]),
 array([0.9964, 0.9972, 0.9972]),
 array([0.9962, 0.9972, 0.997 ]),
 array([0.9964, 0.997 , 0.997 ]),
 array([0.9958, 0.9964, 0.9962]),
 array([0.9952, 0.9966, 0.9962]),
 array([0.9958, 0.996 , 0.9964]),
 array([0.9958, 0.9964, 0.9964]),
 array([0.996 , 0.9962, 0.9964])]

In [85]:
skin_rf_grid.cv_results_

{'mean_fit_time': array([2.2965333 , 2.8455132 , 3.42109098]),
 'std_fit_time': array([0.04453993, 0.10469531, 0.16527318]),
 'mean_score_time': array([0.32059102, 0.3071816 , 0.32239404]),
 'std_score_time': array([0.00986292, 0.00534017, 0.02777124]),
 'param_max_features': masked_array(data=[1, 2, 3],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1024, 1024, 1024],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 1, 'n_estimators': 1024},
  {'max_features': 2, 'n_estimators': 1024},
  {'max_features': 3, 'n_estimators': 1024}],
 'split0_test_accuracy': array([0.994, 0.995, 0.996]),
 'split1_test_accuracy': array([0.997, 0.998, 0.997]),
 'split2_test_accuracy': array([0.996, 0.996, 0.998]),
 'split3_test_accuracy': array([0.997, 0.997, 0.996]),
 'split4_test_accuracy': array([0.996, 0.995, 0.995]),
 'mean_test_accuracy': a

In [86]:
skin_rf_grid.cv_results_['rank_test_accuracy']

array([3, 2, 1])

In [87]:
skin_rf_grid.cv_results_['params'][ np.argmin(skin_rf_grid.cv_results_['rank_test_accuracy'])]

{'max_features': 3, 'n_estimators': 1024}

In [88]:
skin_rf_grid.cv_results_['params'][ np.argmin(skin_rf_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'max_features': 1, 'n_estimators': 1024}

In [89]:
skin_rf_results = pd.DataFrame(skin_rf_grid.cv_results_['params'])
skin_rf_results['score_acc'] = skin_rf_grid.cv_results_['mean_test_accuracy']
skin_rf_cols = skin_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
skin_rf_results.columns = skin_rf_cols
skin_rf_results = skin_rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
skin_rf_results

,max_features,n_estimators,score_acc
0,3,1024,0.9964
1,2,1024,0.9962
2,1,1024,0.9960


In [91]:
X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
skin_rf_best = RandomForestClassifier(n_estimators=1024, criterion='entropy', max_features=3)
skin_rf_best.fit(X_train, y_train)
skin_pred_rf = skin_rf_best.predict(X_test)
print('Random Forest:', classification_report(y_test, skin_pred_rf))

Random Forest:               precision    recall  f1-score   support

           0       0.99      0.99      0.99     49829
           1       1.00      1.00      1.00    190228

    accuracy                           1.00    240057
   macro avg       1.00      1.00      1.00    240057
weighted avg       1.00      1.00      1.00    240057



In [92]:
#Logistic Regression
for i in range(5):
    print('trial', i+1)
    skin_logreg_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', linear_model.LogisticRegression())])
            search_space = [{'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}]
            skin_logreg_grid = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0)
            skin_logreg_grid.fit(X_train, y_train)
            print('mean test accuracy:', skin_logreg_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auc ovr:', skin_logreg_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', skin_logreg_grid.cv_results_['mean_test_f1_micro'])
            skin_logreg_results = pd.DataFrame(skin_logreg_grid.cv_results_['params'])
            skin_logreg_results['score_acc'] = skin_logreg_grid.cv_results_['mean_test_accuracy']
            skin_logreg_list.append(skin_logreg_grid.cv_results_['mean_test_accuracy'])
            skin_logreg_cols = skin_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            skin_logreg_results.columns = skin_logreg_cols
            print(skin_logreg_results)

trial 1
mean test accuracy: [0.7878 0.7878 0.7878 0.7836 0.904  0.9036 0.9174 0.9178 0.9192 0.9192
 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.7878 0.7836
 0.9036 0.9178 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192]
mean test roc auc ovr: [0.5        0.92269169 0.5        0.94564608 0.94569712 0.94624967
 0.94678427 0.94729028 0.9481254  0.94817088 0.94829054 0.94829892
 0.94830849 0.94830969 0.94831448 0.94831208 0.94831448 0.94831208
 0.92267972 0.94564489 0.94625086 0.94729028 0.94817088 0.94830011
 0.94831089 0.94831089 0.94831208 0.94831208 0.94831328]
mean test f1 micro: [0.7878 0.7878 0.7878 0.7836 0.904  0.9036 0.9174 0.9178 0.9192 0.9192
 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.7878 0.7836
 0.9036 0.9178 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7878
1   LogisticRegression(max_iter=5000) 

mean test accuracy: [0.7878 0.7878 0.7878 0.7836 0.904  0.9036 0.9174 0.9178 0.9192 0.9192
 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.7878 0.7836
 0.9036 0.9178 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192]
mean test roc auc ovr: [0.5        0.92267852 0.5        0.94564489 0.94569832 0.94624966
 0.94678427 0.94729147 0.9481254  0.94816968 0.94829053 0.94829891
 0.94830969 0.94831448 0.94831448 0.94831328 0.94831328 0.94831208
 0.92267972 0.94564489 0.94625086 0.94729028 0.94817088 0.94830011
 0.94831089 0.94831089 0.94831208 0.94831208 0.94831328]
mean test f1 micro: [0.7878 0.7878 0.7878 0.7836 0.904  0.9036 0.9174 0.9178 0.9192 0.9192
 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.7878 0.7836
 0.9036 0.9178 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192 0.9192]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7878
1   LogisticRegression(max_iter=5000)      0.0

mean test accuracy: [0.7914 0.7914 0.7914 0.7876 0.9032 0.903  0.9166 0.9168 0.9182 0.918
 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.7914 0.7876
 0.903  0.9168 0.918  0.9184 0.9184 0.9184 0.9184 0.9184 0.9184]
mean test roc auc ovr: [0.5        0.9218382  0.5        0.94614654 0.94521054 0.94608067
 0.947721   0.94887086 0.95057278 0.95060189 0.95086352 0.95086837
 0.9508962  0.9508962  0.9508962  0.95089621 0.95089863 0.9508962
 0.92182247 0.94615017 0.94608431 0.94887086 0.9506031  0.95086595
 0.9508962  0.9508962  0.9508962  0.9508962  0.95089742]
mean test f1 micro: [0.7914 0.7914 0.7914 0.7876 0.9032 0.903  0.9166 0.9168 0.9182 0.918
 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.7914 0.7876
 0.903  0.9168 0.918  0.9184 0.9184 0.9184 0.9184 0.9184 0.9184]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7914
1   LogisticRegression(max_iter=5000)      0.0001

mean test accuracy: [0.7914 0.7914 0.7914 0.7876 0.9032 0.903  0.9166 0.9168 0.9182 0.918
 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.7914 0.7876
 0.903  0.9168 0.918  0.9184 0.9184 0.9184 0.9184 0.9184 0.9184]
mean test roc auc ovr: [0.5        0.92181763 0.5        0.94615138 0.94520932 0.94607702
 0.94771858 0.94887329 0.95057157 0.95060189 0.95086109 0.95086716
 0.9508962  0.95089499 0.95089499 0.95089742 0.9508962  0.95089862
 0.92182247 0.94615017 0.94608431 0.94887086 0.9506031  0.95086595
 0.9508962  0.9508962  0.9508962  0.9508962  0.95089742]
mean test f1 micro: [0.7914 0.7914 0.7914 0.7876 0.9032 0.903  0.9166 0.9168 0.9182 0.918
 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.9184 0.7914 0.7876
 0.903  0.9168 0.918  0.9184 0.9184 0.9184 0.9184 0.9184 0.9184]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7914
1   LogisticRegression(max_iter=5000)      0.000

mean test accuracy: [0.7984 0.7984 0.7984 0.7968 0.9054 0.9056 0.9224 0.9224 0.9238 0.9238
 0.924  0.924  0.924  0.924  0.924  0.924  0.924  0.924  0.7984 0.7968
 0.9056 0.9224 0.9238 0.924  0.924  0.924  0.924  0.924  0.924 ]
mean test roc auc ovr: [0.5        0.92093674 0.5        0.94659298 0.94761301 0.95022117
 0.95550652 0.95547789 0.95762571 0.95754245 0.95786548 0.95785554
 0.95789158 0.95789158 0.95789407 0.95789531 0.95789531 0.95789406
 0.92086849 0.9465967  0.95022116 0.95547789 0.95754369 0.9578543
 0.95789282 0.95789407 0.95789655 0.95789655 0.95789407]
mean test f1 micro: [0.7984 0.7984 0.7984 0.7968 0.9054 0.9056 0.9224 0.9224 0.9238 0.9238
 0.924  0.924  0.924  0.924  0.924  0.924  0.924  0.924  0.7984 0.7968
 0.9056 0.9224 0.9238 0.924  0.924  0.924  0.924  0.924  0.924 ]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7984
1   LogisticRegression(max_iter=5000)      0.00

...

mean test accuracy: [0.7886 0.7886 0.7886 0.7852 0.9004 0.8992 0.9132 0.913  0.9138 0.9136
 0.9138 0.9138 0.914  0.914  0.914  0.914  0.914  0.914  0.7886 0.7852
 0.8992 0.913  0.9136 0.9138 0.914  0.914  0.914  0.914  0.9138]
mean test roc auc ovr: [0.5        0.92160449 0.5        0.94473717 0.94385357 0.94449649
 0.94510675 0.94570058 0.94640247 0.94645049 0.94655126 0.94657288
 0.94658368 0.94659329 0.94658849 0.94658969 0.94658848 0.94659089
 0.92161407 0.94474077 0.94449649 0.94570058 0.9464493  0.94657048
 0.94658848 0.94658969 0.94658969 0.94658969 0.94659209]
mean test f1 micro: [0.7886 0.7886 0.7886 0.7852 0.9004 0.8992 0.9132 0.913  0.9138 0.9136
 0.9138 0.9138 0.914  0.914  0.914  0.914  0.914  0.914  0.7886 0.7852
 0.8992 0.913  0.9136 0.9138 0.914  0.914  0.914  0.914  0.9138]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7886
1   LogisticRegression(max_iter=5000)      0.0

mean test accuracy: [0.7886 0.7886 0.7886 0.7852 0.9004 0.8992 0.9132 0.913  0.9138 0.9136
 0.9138 0.9138 0.914  0.914  0.914  0.914  0.914  0.914  0.7886 0.7852
 0.8992 0.913  0.9136 0.9138 0.914  0.914  0.914  0.914  0.914 ]
mean test roc auc ovr: [0.5        0.92161531 0.5        0.94475395 0.94385958 0.94449409
 0.94510675 0.94569938 0.94640126 0.9464493  0.94656928 0.94656928
 0.94658728 0.94659089 0.94658968 0.94658969 0.94658608 0.94658969
 0.92161407 0.94474077 0.94449649 0.94570058 0.9464493  0.94657048
 0.94658848 0.94658969 0.94658969 0.94658969 0.94658849]
mean test f1 micro: [0.7886 0.7886 0.7886 0.7852 0.9004 0.8992 0.9132 0.913  0.9138 0.9136
 0.9138 0.9138 0.914  0.914  0.914  0.914  0.914  0.914  0.7886 0.7852
 0.8992 0.913  0.9136 0.9138 0.914  0.914  0.914  0.914  0.914 ]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7886
1   LogisticRegression(max_iter=5000)      0.0

mean test accuracy: [0.7916 0.7916 0.7916 0.788  0.9032 0.9034 0.9168 0.9168 0.9168 0.917
 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.7916 0.788
 0.9034 0.9168 0.917  0.9172 0.9172 0.9172 0.9172 0.9172 0.9172]
mean test roc auc ovr: [0.5        0.92517892 0.5        0.94734425 0.94621675 0.94745356
 0.94962468 0.95009097 0.95148549 0.95152538 0.95173145 0.95173265
 0.95175085 0.95175207 0.95175085 0.95174721 0.95175206 0.95174842
 0.92518014 0.94734304 0.94745357 0.95008976 0.9515266  0.95173265
 0.95175207 0.95174964 0.95175085 0.95175085 0.95175085]
mean test f1 micro: [0.7916 0.7916 0.7916 0.788  0.9032 0.9034 0.9168 0.9168 0.9168 0.917
 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.9172 0.7916 0.788
 0.9034 0.9168 0.917  0.9172 0.9172 0.9172 0.9172 0.9172 0.9172]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7916
1   LogisticRegression(max_iter=5000)      0.0001 

mean test accuracy: [0.7958 0.7958 0.7958 0.7912 0.9016 0.901  0.919  0.919  0.9192 0.9192
 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.7958 0.7912
 0.901  0.919  0.9192 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188]
mean test roc auc ovr: [0.5        0.92510191 0.5        0.94825697 0.94721473 0.94790832
 0.94860772 0.94951384 0.95034726 0.95041128 0.95056265 0.95056758
 0.95057744 0.95057867 0.95058236 0.9505799  0.9505799  0.95058852
 0.92509822 0.94825697 0.94790832 0.94950891 0.95041128 0.95057374
 0.95057867 0.95058236 0.95058236 0.95058236 0.95058483]
mean test f1 micro: [0.7958 0.7958 0.7958 0.7912 0.9016 0.901  0.919  0.919  0.9192 0.9192
 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.7958 0.7912
 0.901  0.919  0.9192 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7958
1   LogisticRegression(max_iter=5000)      0.0

mean test accuracy: [0.7958 0.7958 0.7958 0.7912 0.9016 0.901  0.919  0.919  0.9192 0.9192
 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.7958 0.7912
 0.901  0.919  0.9192 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188]
mean test roc auc ovr: [0.5        0.92509822 0.5        0.94825943 0.94721473 0.94790955
 0.94860772 0.94951014 0.95034602 0.95041128 0.95055896 0.95057128
 0.95057744 0.95057744 0.95058236 0.95058236 0.95058113 0.95058359
 0.92509822 0.94825697 0.94790832 0.94950891 0.95041128 0.95057374
 0.95057867 0.95058236 0.95058236 0.95058236 0.95058236]
mean test f1 micro: [0.7958 0.7958 0.7958 0.7912 0.9016 0.901  0.919  0.919  0.9192 0.9192
 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188 0.7958 0.7912
 0.901  0.919  0.9192 0.9188 0.9188 0.9188 0.9188 0.9188 0.9188]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.7958
1   LogisticRegression(max_iter=5000)      0.0

In [93]:
skin_logreg_list

[array([0.7942, 0.7942, 0.7942, 0.791 , 0.9092, 0.9104, 0.921 , 0.9214,
        0.9236, 0.9236, 0.924 , 0.924 , 0.924 , 0.924 , 0.924 , 0.924 ,
        0.924 , 0.924 , 0.7942, 0.791 , 0.9104, 0.9214, 0.9236, 0.924 ,
        0.924 , 0.924 , 0.924 , 0.924 , 0.924 ]),
 array([0.7942, 0.7942, 0.7942, 0.791 , 0.9092, 0.9104, 0.921 , 0.9214,
        0.9236, 0.9236, 0.924 , 0.924 , 0.924 , 0.924 , 0.924 , 0.924 ,
        0.924 , 0.924 , 0.7942, 0.791 , 0.9104, 0.9214, 0.9236, 0.924 ,
        0.924 , 0.924 , 0.924 , 0.924 , 0.924 ]),
 array([0.7942, 0.7942, 0.7942, 0.791 , 0.9092, 0.9104, 0.921 , 0.9214,
        0.9236, 0.9236, 0.924 , 0.924 , 0.924 , 0.924 , 0.924 , 0.924 ,
        0.924 , 0.924 , 0.7942, 0.791 , 0.9104, 0.9214, 0.9236, 0.924 ,
        0.924 , 0.924 , 0.924 , 0.924 , 0.924 ]),
 array([0.7942, 0.7942, 0.7942, 0.791 , 0.9092, 0.9104, 0.921 , 0.9214,
        0.9236, 0.9236, 0.924 , 0.924 , 0.924 , 0.924 , 0.924 , 0.924 ,
        0.924 , 0.924 , 0.7942, 0.791 , 0.9104, 0.9214, 0.

In [94]:
skin_logreg_grid.cv_results_

{'mean_fit_time': array([0.00579939, 0.0097991 , 0.00560536, 0.01020207, 0.01099925,
        0.01079855, 0.01119676, 0.01020551, 0.0122014 , 0.01199851,
        0.01139941, 0.01179914, 0.01159887, 0.01059923, 0.01159997,
        0.01179981, 0.0123991 , 0.01079659, 0.01019912, 0.00839858,
        0.00899897, 0.01019869, 0.01019893, 0.0103982 , 0.00939946,
        0.00999951, 0.0101985 , 0.00959921, 0.01119881]),
 'std_fit_time': array([0.00074823, 0.00039976, 0.00048782, 0.00039872, 0.00063249,
        0.00039976, 0.00098349, 0.00039678, 0.00039675, 0.00209528,
        0.00149697, 0.0003989 , 0.00101971, 0.00049024, 0.00049043,
        0.00074903, 0.00048967, 0.00039846, 0.00097679, 0.00101925,
        0.00063264, 0.00097843, 0.00039954, 0.00079916, 0.00048984,
        0.00063211, 0.00039749, 0.00049008, 0.00116597]),
 'mean_score_time': array([0.00600748, 0.00500331, 0.00579405, 0.0083951 , 0.00540061,
        0.00600057, 0.00520601, 0.00499396, 0.00899792, 0.00699997,
        0.0052  

In [98]:
skin_logreg_grid.cv_results_['rank_test_accuracy']

array([24, 24, 24, 28, 21, 22,  4,  4,  1,  1,  7,  7,  7,  7,  7,  7,  7,
        7, 24, 28, 22,  4,  1,  7,  7,  7,  7,  7,  7])

In [95]:
skin_logreg_grid.cv_results_['params'][ np.argmin(skin_logreg_grid.cv_results_['rank_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [96]:
skin_logreg_grid.cv_results_['params'][ np.argmin(skin_logreg_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__penalty': 'none',
 'classifier__solver': 'saga'}

In [97]:
skin_logreg_results = pd.DataFrame(skin_logreg_grid.cv_results_['params'])
skin_logreg_results['score_acc'] = skin_logreg_grid.cv_results_['mean_test_accuracy']
skin_logreg_cols = skin_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
skin_logreg_results.columns = skin_logreg_cols
skin_logreg_results = skin_logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
skin_logreg_results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=5000),1.0000,l1,saga,0.9192
1,LogisticRegression(max_iter=5000),1.0000,l2,saga,0.9192
2,LogisticRegression(max_iter=5000),1.0000,l2,lbfgs,0.9192
3,LogisticRegression(max_iter=5000),0.1000,l1,saga,0.9190
4,LogisticRegression(max_iter=5000),0.1000,l2,saga,0.9190
5,LogisticRegression(max_iter=5000),0.1000,l2,lbfgs,0.9190
6,LogisticRegression(max_iter=5000),1000.0000,l1,saga,0.9188
7,LogisticRegression(max_iter=5000),1000.0000,l2,saga,0.9188
8,LogisticRegression(max_iter=5000),NaN,none,lbfgs,0.9188
9,LogisticRegression(max_iter=5000),10000.0000,l2,lbfgs,0.9188


In [99]:
X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
skin_logreg_best = linear_model.LogisticRegression(penalty='none', solver='saga', max_iter=5000)
skin_logreg_best.fit(X_train, y_train)
skin_pred_logreg = skin_logreg_best.predict(X_test)
print('Logistic Regression:', classification_report(y_test, skin_pred_logreg))

Logistic Regression:               precision    recall  f1-score   support

           0       0.79      0.82      0.80     49799
           1       0.95      0.94      0.95    190258

    accuracy                           0.92    240057
   macro avg       0.87      0.88      0.88    240057
weighted avg       0.92      0.92      0.92    240057



In [100]:
# KNN
for i in range(5):
    print('trial', i+1)
    skin_knn_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            skin_knn = KNeighborsClassifier(weights='distance')
            k_list = [1, 5 , 9 , 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101]
            skin_knn_grid = GridSearchCV(skin_knn, {'n_neighbors':k_list}, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0)
            skin_knn_grid.fit(X_train, y_train)
            print('mean test accuracy:', skin_knn_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', skin_knn_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', skin_knn_grid.cv_results_['mean_test_f1_micro'])
            skin_knn_results = pd.DataFrame(skin_knn_grid.cv_results_['params'])
            skin_knn_results['score_acc'] = skin_knn_grid.cv_results_['mean_test_accuracy']
            skin_knn_list.append(skin_knn_grid.cv_results_['mean_test_accuracy'])
            skin_knn_cols = skin_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            skin_knn_results.columns = skin_knn_cols
            skin_knn_results = skin_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(skin_knn_results)

trial 1
mean test accuracy: [0.9982 0.9974 0.9966 0.9962 0.996  0.9952 0.9948 0.9942 0.994  0.9936
 0.9934 0.993  0.9924 0.9916 0.9914 0.991  0.9906 0.99   0.989  0.9884
 0.9872 0.9868 0.9864 0.986  0.9854 0.985 ]
mean test roc auo ovr: [0.99886507 0.99910509 0.99933474 0.99945048 0.99967952 0.99965759
 0.99964175 0.99962712 0.99961491 0.99960272 0.99970871 0.99970445
 0.99969956 0.99968675 0.99966723 0.99966053 0.99965199 0.99963856
 0.99961663 0.99959772 0.99958675 0.99956726 0.9995563  0.99954839
 0.99953621 0.99952646]
mean test f1 micro: [0.9982 0.9974 0.9966 0.9962 0.996  0.9952 0.9948 0.9942 0.994  0.9936
 0.9934 0.993  0.9924 0.9916 0.9914 0.991  0.9906 0.99   0.989  0.9884
 0.9872 0.9868 0.9864 0.986  0.9854 0.985 ]
    n_neighbors  score_acc
0             1     0.9982
1             5     0.9974
2             9     0.9966
3            13     0.9962
4            17     0.9960
5            21     0.9952
6            25     0.9948
7            29     0.9942
8            33     0.

mean test accuracy: [0.9982 0.9974 0.9972 0.9968 0.9966 0.9958 0.995  0.9948 0.9936 0.9932
 0.9922 0.992  0.9912 0.9904 0.9894 0.9884 0.9876 0.9868 0.9866 0.9856
 0.9848 0.9842 0.9838 0.9834 0.983  0.9828]
mean test roc auo ovr: [0.99850637 0.99910631 0.99934431 0.99932843 0.99943927 0.99955727
 0.99955361 0.99954444 0.99953528 0.9995298  0.99951822 0.99951578
 0.99950785 0.99949628 0.99949139 0.99948893 0.999481   0.99947918
 0.9994804  0.99957482 0.99956629 0.99956384 0.99957114 0.99956505
 0.99955651 0.99954553]
mean test f1 micro: [0.9982 0.9974 0.9972 0.9968 0.9966 0.9958 0.995  0.9948 0.9936 0.9932
 0.9922 0.992  0.9912 0.9904 0.9894 0.9884 0.9876 0.9868 0.9866 0.9856
 0.9848 0.9842 0.9838 0.9834 0.983  0.9828]
    n_neighbors  score_acc
0             1     0.9982
1             5     0.9974
2             9     0.9972
3            13     0.9968
4            17     0.9966
5            21     0.9958
6            25     0.9950
7            29     0.9948
8            33     0.9936
9  

mean test accuracy: [0.9978 0.997  0.997  0.9966 0.9962 0.9954 0.9948 0.9942 0.994  0.994
 0.9934 0.9926 0.992  0.9914 0.9908 0.9896 0.989  0.9888 0.988  0.9876
 0.987  0.9864 0.9862 0.9858 0.9848 0.9846]
mean test roc auo ovr: [0.99789168 0.9989822  0.99933995 0.9993222  0.99930933 0.99930014
 0.99928175 0.99950776 0.99949429 0.99948633 0.99948387 0.9994753
 0.99946305 0.99945877 0.99944836 0.99943918 0.99942509 0.99941958
 0.99941346 0.9994012  0.99939262 0.99937977 0.99937058 0.99935157
 0.99934543 0.99933439]
mean test f1 micro: [0.9978 0.997  0.997  0.9966 0.9962 0.9954 0.9948 0.9942 0.994  0.994
 0.9934 0.9926 0.992  0.9914 0.9908 0.9896 0.989  0.9888 0.988  0.9876
 0.987  0.9864 0.9862 0.9858 0.9848 0.9846]
    n_neighbors  score_acc
0             1     0.9978
1             5     0.9970
2             9     0.9970
3            13     0.9966
4            17     0.9962
5            21     0.9954
6            25     0.9948
7            29     0.9942
8            33     0.9940
9     

mean test accuracy: [0.9982 0.998  0.9978 0.9958 0.9946 0.9936 0.9922 0.9918 0.9914 0.9906
 0.9904 0.9894 0.9896 0.9896 0.9892 0.988  0.9878 0.9874 0.9874 0.987
 0.9864 0.9854 0.9852 0.985  0.9842 0.9844]
mean test roc auo ovr: [0.9981543  0.99922754 0.9993447  0.99933438 0.99931071 0.99942059
 0.99953168 0.99950985 0.99949955 0.99948924 0.99960396 0.99960397
 0.99959062 0.99958457 0.99957729 0.99956273 0.99954696 0.99952998
 0.99951662 0.99950327 0.99948872 0.99946931 0.99945838 0.99944746
 0.99943533 0.99941592]
mean test f1 micro: [0.9982 0.998  0.9978 0.9958 0.9946 0.9936 0.9922 0.9918 0.9914 0.9906
 0.9904 0.9894 0.9896 0.9896 0.9892 0.988  0.9878 0.9874 0.9874 0.987
 0.9864 0.9854 0.9852 0.985  0.9842 0.9844]
    n_neighbors  score_acc
0             1     0.9982
1             5     0.9980
2             9     0.9978
3            13     0.9958
4            17     0.9946
5            21     0.9936
6            25     0.9922
7            29     0.9918
8            33     0.9914
9    

mean test accuracy: [0.9974 0.9974 0.997  0.9968 0.9966 0.995  0.9938 0.993  0.992  0.9904
 0.989  0.9886 0.988  0.9876 0.9866 0.9864 0.986  0.9854 0.985  0.9844
 0.984  0.9836 0.983  0.9828 0.9828 0.9828]
mean test roc auo ovr: [0.99764437 0.99910907 0.99921908 0.99920255 0.99931256 0.99942059
 0.9995427  0.9995231  0.99951454 0.99950472 0.99949492 0.9994784
 0.99946432 0.99945147 0.99943615 0.99941041 0.99939265 0.99937305
 0.99947146 0.99947634 0.99946776 0.99956044 0.99965211 0.9996607
 0.99965212 0.99964355]
mean test f1 micro: [0.9974 0.9974 0.997  0.9968 0.9966 0.995  0.9938 0.993  0.992  0.9904
 0.989  0.9886 0.988  0.9876 0.9866 0.9864 0.986  0.9854 0.985  0.9844
 0.984  0.9836 0.983  0.9828 0.9828 0.9828]
    n_neighbors  score_acc
0             1     0.9974
1             5     0.9974
2             9     0.9970
3            13     0.9968
4            17     0.9966
5            21     0.9950
6            25     0.9938
7            29     0.9930
8            33     0.9920
9    

...

mean test accuracy: [0.9982 0.9972 0.9964 0.9958 0.9954 0.9946 0.9938 0.9934 0.9932 0.993
 0.9918 0.9912 0.9902 0.9888 0.9874 0.9872 0.987  0.9866 0.9862 0.9854
 0.9852 0.9842 0.9832 0.9824 0.9818 0.9816]
mean test roc auo ovr: [0.99885899 0.99910408 0.99922182 0.99933715 0.99932036 0.99930535
 0.99928676 0.99940572 0.99951703 0.99951281 0.99949842 0.99948643
 0.99958735 0.99968368 0.99969089 0.99966927 0.99965187 0.99962907
 0.99973479 0.99971799 0.99971079 0.99970837 0.99969157 0.99967717
 0.99966637 0.99966517]
mean test f1 micro: [0.9982 0.9972 0.9964 0.9958 0.9954 0.9946 0.9938 0.9934 0.9932 0.993
 0.9918 0.9912 0.9902 0.9888 0.9874 0.9872 0.987  0.9866 0.9862 0.9854
 0.9852 0.9842 0.9832 0.9824 0.9818 0.9816]
    n_neighbors  score_acc
0             1     0.9982
1             5     0.9972
2             9     0.9964
3            13     0.9958
4            17     0.9954
5            21     0.9946
6            25     0.9938
7            29     0.9934
8            33     0.9932
9    

mean test accuracy: [0.9976 0.9974 0.9968 0.9958 0.9948 0.9948 0.9948 0.9942 0.9936 0.9926
 0.9926 0.9918 0.9918 0.9916 0.9914 0.9906 0.9906 0.9904 0.99   0.9892
 0.9888 0.9888 0.9888 0.9878 0.9874 0.987 ]
mean test roc auo ovr: [0.99736902 0.99911433 0.99921344 0.99932383 0.99942733 0.99941478
 0.9994016  0.9993878  0.99936835 0.99934449 0.99932188 0.9993087
 0.99929489 0.99940152 0.99940338 0.99939084 0.99937891 0.99936759
 0.99935567 0.99934185 0.99942778 0.99953192 0.99952625 0.99950927
 0.99950362 0.99948665]
mean test f1 micro: [0.9976 0.9974 0.9968 0.9958 0.9948 0.9948 0.9948 0.9942 0.9936 0.9926
 0.9926 0.9918 0.9918 0.9916 0.9914 0.9906 0.9906 0.9904 0.99   0.9892
 0.9888 0.9888 0.9888 0.9878 0.9874 0.987 ]
    n_neighbors  score_acc
0             1     0.9976
1             5     0.9974
2             9     0.9968
3            13     0.9958
4            17     0.9948
5            21     0.9948
6            25     0.9948
7            29     0.9942
8            33     0.9936
9   

mean test accuracy: [0.9986 0.9966 0.9952 0.9954 0.9948 0.9942 0.9936 0.993  0.9924 0.9912
 0.9902 0.9902 0.9896 0.989  0.9886 0.9876 0.9872 0.9866 0.9862 0.9858
 0.9854 0.9846 0.9844 0.984  0.9834 0.9832]
mean test roc auo ovr: [0.99875937 0.99947924 0.99970909 0.9996999  0.9996809  0.99978954
 0.99977543 0.99987626 0.99985297 0.99983949 0.99981619 0.99980764
 0.99977944 0.99977209 0.99975128 0.99972553 0.99970714 0.99968139
 0.99965688 0.99963606 0.99961768 0.99959931 0.99957361 0.99956259
 0.99954054 0.99953074]
mean test f1 micro: [0.9986 0.9966 0.9952 0.9954 0.9948 0.9942 0.9936 0.993  0.9924 0.9912
 0.9902 0.9902 0.9896 0.989  0.9886 0.9876 0.9872 0.9866 0.9862 0.9858
 0.9854 0.9846 0.9844 0.984  0.9834 0.9832]
    n_neighbors  score_acc
0             1     0.9986
1             5     0.9966
2            13     0.9954
3             9     0.9952
4            17     0.9948
5            21     0.9942
6            25     0.9936
7            29     0.9930
8            33     0.9924
9  

mean test accuracy: [0.999  0.9976 0.9968 0.9956 0.9956 0.995  0.995  0.994  0.9936 0.9918
 0.991  0.9904 0.9898 0.9894 0.9892 0.9882 0.987  0.9864 0.9844 0.984
 0.9826 0.9814 0.9804 0.9796 0.9796 0.9792]
mean test roc auo ovr: [0.99936869 0.99935837 0.99946945 0.99969831 0.99981243 0.99979422
 0.99978511 0.99976689 0.99975596 0.99973836 0.99973168 0.99971894
 0.99971104 0.9996989  0.99967522 0.99966429 0.99964548 0.99963697
 0.99962544 0.99962422 0.99962665 0.99959933 0.99957564 0.99957565
 0.99956168 0.99954225]
mean test f1 micro: [0.999  0.9976 0.9968 0.9956 0.9956 0.995  0.995  0.994  0.9936 0.9918
 0.991  0.9904 0.9898 0.9894 0.9892 0.9882 0.987  0.9864 0.9844 0.984
 0.9826 0.9814 0.9804 0.9796 0.9796 0.9792]
    n_neighbors  score_acc
0             1     0.9990
1             5     0.9976
2             9     0.9968
3            13     0.9956
4            17     0.9956
5            21     0.9950
6            25     0.9950
7            29     0.9940
8            33     0.9936
9    

mean test accuracy: [0.998  0.9978 0.9978 0.9972 0.9964 0.9952 0.9948 0.9942 0.9934 0.9926
 0.992  0.9914 0.99   0.9896 0.9888 0.9878 0.987  0.9862 0.9848 0.9846
 0.9848 0.9844 0.9844 0.984  0.9834 0.983 ]
mean test roc auo ovr: [0.99801117 0.99910703 0.99922136 0.99946303 0.99944259 0.99942649
 0.99941661 0.99940116 0.99938508 0.99937087 0.99947841 0.99946541
 0.99945428 0.99944005 0.99942954 0.99941532 0.99939862 0.99949916
 0.9995029  0.99949175 0.99947257 0.99946638 0.99944906 0.99944225
 0.99942431 0.99940822]
mean test f1 micro: [0.998  0.9978 0.9978 0.9972 0.9964 0.9952 0.9948 0.9942 0.9934 0.9926
 0.992  0.9914 0.99   0.9896 0.9888 0.9878 0.987  0.9862 0.9848 0.9846
 0.9848 0.9844 0.9844 0.984  0.9834 0.983 ]
    n_neighbors  score_acc
0             1     0.9980
1             5     0.9978
2             9     0.9978
3            13     0.9972
4            17     0.9964
5            21     0.9952
6            25     0.9948
7            29     0.9942
8            33     0.9934
9  

In [101]:
skin_knn_list

[array([0.9976, 0.9974, 0.9968, 0.9958, 0.9948, 0.9948, 0.9948, 0.9942,
        0.9936, 0.9926, 0.9926, 0.9918, 0.9918, 0.9916, 0.9914, 0.9906,
        0.9906, 0.9904, 0.99  , 0.9892, 0.9888, 0.9888, 0.9888, 0.9878,
        0.9874, 0.987 ]),
 array([0.9976, 0.9974, 0.9968, 0.9958, 0.9948, 0.9948, 0.9948, 0.9942,
        0.9936, 0.9926, 0.9926, 0.9918, 0.9918, 0.9916, 0.9914, 0.9906,
        0.9906, 0.9904, 0.99  , 0.9892, 0.9888, 0.9888, 0.9888, 0.9878,
        0.9874, 0.987 ]),
 array([0.9976, 0.9974, 0.9968, 0.9958, 0.9948, 0.9948, 0.9948, 0.9942,
        0.9936, 0.9926, 0.9926, 0.9918, 0.9918, 0.9916, 0.9914, 0.9906,
        0.9906, 0.9904, 0.99  , 0.9892, 0.9888, 0.9888, 0.9888, 0.9878,
        0.9874, 0.987 ]),
 array([0.9976, 0.9974, 0.9968, 0.9958, 0.9948, 0.9948, 0.9948, 0.9942,
        0.9936, 0.9926, 0.9926, 0.9918, 0.9918, 0.9916, 0.9914, 0.9906,
        0.9906, 0.9904, 0.99  , 0.9892, 0.9888, 0.9888, 0.9888, 0.9878,
        0.9874, 0.987 ]),
 array([0.9976, 0.9974, 0.9968, 

In [102]:
skin_knn_grid.cv_results_

{'mean_fit_time': array([0.00499916, 0.00539966, 0.00519924, 0.00519948, 0.00539951,
        0.00519967, 0.00499969, 0.00539675, 0.00499964, 0.00539961,
        0.00499921, 0.00499868, 0.00559912, 0.00519929, 0.00539899,
        0.00559878, 0.00520058, 0.0053997 , 0.00519958, 0.00519924,
        0.00519886, 0.00499926, 0.00540118, 0.00640059, 0.0050004 ,
        0.00499926]),
 'std_fit_time': array([5.00111031e-07, 4.89823585e-04, 7.48379360e-04, 4.00519666e-04,
        4.89551096e-04, 4.00185766e-04, 6.32409753e-04, 4.86947675e-04,
        2.61174468e-07, 4.89959835e-04, 6.32711332e-04, 1.01152436e-06,
        7.99465222e-04, 3.99661217e-04, 4.89687269e-04, 4.89201066e-04,
        4.01997690e-04, 4.90563292e-04, 3.99994918e-04, 4.00044022e-04,
        3.99160513e-04, 6.50319180e-07, 4.92089303e-04, 2.80249121e-03,
        6.32113409e-04, 6.31957713e-04]),
 'mean_score_time': array([0.01100054, 0.01199951, 0.01500015, 0.01739988, 0.01739988,
        0.01759925, 0.02019968, 0.02080235, 

In [103]:
skin_knn_grid.cv_results_['rank_test_accuracy']

array([ 1,  2,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 19, 22, 22, 24, 25, 26])

In [104]:
skin_knn_grid.cv_results_['params'][ np.argmin(skin_knn_grid.cv_results_['rank_test_accuracy'])]

{'n_neighbors': 1}

In [105]:
skin_knn_grid.cv_results_['params'][ np.argmin(skin_knn_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'n_neighbors': 73}

In [107]:
skin_knn_grid.cv_results_['params'][ np.argmin(skin_knn_grid.cv_results_['rank_test_f1_micro'])]

{'n_neighbors': 1}

In [106]:
skin_knn_results = pd.DataFrame(skin_knn_grid.cv_results_['params'])
skin_knn_results['score_acc'] = skin_knn_grid.cv_results_['mean_test_accuracy']
skin_knn_cols = skin_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
skin_knn_results.columns = skin_knn_cols
skin_knn_results = skin_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
skin_knn_results

,n_neighbors,score_acc
0,1,0.9980
1,5,0.9978
2,9,0.9978
3,13,0.9972
4,17,0.9964
5,21,0.9952
6,25,0.9948
7,29,0.9942
8,33,0.9934
9,37,0.9926


In [108]:
X_train, X_test, y_train, y_test = train_test_split(skin_data2, skin_data['decision'], train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
skin_knn_best = KNeighborsClassifier(weights='distance', n_neighbors=73)
skin_knn_best.fit(X_train, y_train)
skin_pred_knn = skin_knn_best.predict(X_test)
print('KNN:', classification_report(y_test, skin_pred_knn))

KNN:               precision    recall  f1-score   support

           0       0.96      1.00      0.98     49881
           1       1.00      0.99      0.99    190176

    accuracy                           0.99    240057
   macro avg       0.98      0.99      0.99    240057
weighted avg       0.99      0.99      0.99    240057



In [ ]:
stats.ttest_ind()